***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-2e86  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  6 12:48 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '318305570_207757535_bucket'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if not ((b.name.startswith('index_posting_locs_title/') or b.name.startswith('index_posting_locs_title_not_stemmed/') or b.name.startswith('index_posting_locs_training/') or b.name.startswith('index_posting_locs_body/')) and (b.name.endswith('.bin') or b.name.endswith('.pickle') or b.name.endswith('.pkl')) or b.name == 'graphframes.sh' or b.name == 'page_rank_res.csv' or b.name == 'pageviews-202108-user.pkl' or b.name == 'page_rank_norm.pkl' or b.name == 'page_view_norm.pkl'):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [21]:
parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs = parquetFile.limit(2).select("anchor_text", "id").rdd
print(doc_anchor_pairs.take(2))

[Row(anchor_text=[Row(id=884918, text='Miss USA'), Row(id=95198, text='Beauty pageant'), Row(id=8569916, text='English'), Row(id=47737, text='Las Vegas, Nevada'), Row(id=194034, text='Miss America'), Row(id=95198, text='pageant'), Row(id=21216, text='Nevada'), Row(id=194034, text='Miss America'), Row(id=26388, text='Reno'), Row(id=47737, text='Las Vegas'), Row(id=124417, text='Elko'), Row(id=7441, text='Carson City'), Row(id=124372, text='Mesquite'), Row(id=18463, text='Lions Club'), Row(id=1586195, text='Soroptimist'), Row(id=124399, text='Gardnerville'), Row(id=47737, text='Las Vegas, Nevada'), Row(id=194034, text='Miss America'), Row(id=2267715, text='Mohegan Sun'), Row(id=315583, text='Uncasville, Connecticut'), Row(id=41139373, text='Teresa Benitez'), Row(id=41139373, text='Teresa Benitez'), Row(id=15410587, text='Preliminary Lifestyle and Fitness'), Row(id=15410587, text='Preliminary Talent'), Row(id=41139373, text='Teresa Benitez'), Row(id=15410587, text='Quality of Life Award')

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [76]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_anchor_gcp.py

inverted_index_anchor_gcp.py


In [77]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_anchor_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/06 18:23:10 WARN SparkContext: The path /home/dataproc/inverted_index_anchor_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [78]:
from inverted_index_anchor_gcp import *

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [74]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?[\w,]?[\w.]?(?:['\-]?[\w,]?[\w])){0,24}""", re.UNICODE)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


def word_count(text, id):
  ''' Count the frequency of each word in text (tf) that is not included in 
  all_stopwords and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  print(text)
  print(id)
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [stemmer.stem(x) for x in tokens if x not in all_stopwords]
  word_freq = {}
  for token in tokens:
      if token not in all_stopwords:
          if token in word_freq:
              word_freq[token] += 1
          else:
              word_freq[token] = 1
  return [(token, (id, freq)) for token, freq in word_freq.items()]


# english_stopwords = frozenset(stopwords.words('english'))
# corpus_stopwords = ["category", "references", "also", "external", "links",
#                     "may", "first", "see", "history", "people", "one", "two",
#                     "part", "thumb", "including", "second", "following",
#                     "many", "however", "would", "became"]

# all_stopwords = english_stopwords.union(corpus_stopwords)
# RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

# def stemming(tokens):
#     stemmer = PorterStemmer()
#     stemmed_tokens = [stemmer.stem(x) for x in tokens if x not in all_stopwords]
#     return stemmed_tokens


# NUM_BUCKETS = 124
# def token2bucket_id(token):
#     return int(_hash(token),16) % NUM_BUCKETS

# def word_count(text, id):
#     ''' Count the frequency of each word in `text` (tf) that is not included in
#     `all_stopwords` and return entries that will go into our posting lists.
#     Parameters:
#     -----------
#     text: str
#       Text of one document
#     id: int
#       Document id
#     Returns:
#     --------
#     List of tuples
#       A list of (token, (doc_id, tf)) pairs
#       for example: [("Anarchism", (12, 5)), ...]
#     '''
#     tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#     tokens = [stemmer.stem(x) for x in tokens if x not in all_stopwords]
#     word_freq = {}
#     for token in tokens:
#       if token not in all_stopwords:
#           if token in word_freq:
#               word_freq[token] += 1
#           else:
#               word_freq[token] = 1
#     return [(token, (id, freq)) for token, freq in word_freq.items()]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # Sort the unsorted posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[1], reverse = True)
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # Count the number of documents each token appears in
  token_df = postings.map(lambda x: (x[0], len(x[1]))).distinct()
  return token_df

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # Partition postings into buckets
  bucketed_postings = postings.groupBy(lambda x: token2bucket_id(x[0]))

  # Write posting lists to disk and get posting locations for each bucket
  posting_locations = bucketed_postings.map(lambda x: InvertedIndex.write_a_posting_list((x[0], list(x[1])),'index_posting_locs_anchor_text', bucket_name))

  return posting_locations

In [79]:
def remove_id_anchor(pair):
    return [(tup[1], tup[0]) for tup in pair[0]]

def merge_tuples(lst):
    counter = {}
    for tup in lst:
        if tup[0] not in counter:
            counter[tup[0]] = 0
        counter[tup[0]] += 1
    return [(key, counter[key]) for key in counter.keys()]

anchor_pairs = doc_anchor_pairs.flatMap(lambda pair: remove_id_anchor(pair))
anchor_word_counts = anchor_pairs.flatMap(lambda x: word_count(x[0],x[1]))
anchor_postings = anchor_word_counts.groupByKey().mapValues(reduce_word_counts)
anchor_grouped_postings = anchor_postings.map(lambda x: (x[0], merge_tuples(x[1])))
w2df = calculate_df(anchor_grouped_postings)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(anchor_grouped_postings).collect()


24/03/06 18:23:23 WARN TaskSetManager: Lost task 0.0 in stage 193.0 (TID 908) (cluster-2e86-w-3.c.irfinalproject-415108.internal executor 14): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_16043/1621593596.py", line 143, in <lambda>
TypeError: InvertedIndex.write_a_posting_list() takes 1 positional argument but 3 were given

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:552)
	at org.apache.s

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 193.0 failed 4 times, most recent failure: Lost task 0.3 in stage 193.0 (TID 911) (cluster-2e86-w-3.c.irfinalproject-415108.internal executor 14): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_16043/1621593596.py", line 143, in <lambda>
TypeError: InvertedIndex.wr

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='index_posting_locs_anchor_text'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

Putting it all together

In [ ]:

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#partition_func
inverted.posting_locs = super_posting_locs
# inverted._posting_list = dict(postings.collect())
##############################################################
##### need to change filtered_doc_text_pairs to doc_text_pairs
##### when testing the whole corpus
##############################################################
# inverted.doc_count = filtered_doc_text_pairs.count()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.id_title_dict = tokens_doc_pairs.collectAsMap()
# write the global stats out
inverted.write_index('.', 'index_anchor_text')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/index_posting_locs_anchor_text/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
temp_index = InvertedIndex.read_index('index_posting_locs_title', 'index_title', bucket_name)
for item in temp_index.df.items():
    print(item)
    break


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
    # Extract edges from pages
    edges = pages.flatMap(extract_edges).distinct()

    # Extract vertices from edges
    vertices = edges.flatMap(lambda edge: ((edge[0],), (edge[1],))).distinct()

    return edges, vertices

def extract_edges(row):
    edges = []
    for anchor_text in row[1]:
          edges.append((row[0], anchor_text[0]))
    return edges

In [ ]:
t_start = time()
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete
the reporting requirements for assignment #3.

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0.011
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.


In [ ]:
#If you have decided to do the bonus task - please copy the code here

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag